# Notebook Lecture 7: Feedback and the Root Locus Method
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces the concept of feedback control and presents a first method for feedback control analysis with the Root Locus method.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

# Learning Objectives
Previously we have discussed how to express the input-output behavior of a system. However, as seen in Lecture 1, the (open and/or closed-loop) system behavior may be undesired (e.g., unstable). Consequentially, in this lecture we introduce feedback control, and then correspondingly design controllers such that the system achieves the control objective.

After completing this material, you should be able to:

- Understand how feedback control modifies system behavior:
 	- Stabilizing unstable systems
 	- Improving responsiveness
 	- Reducing oscillations

- Apply the Root Locus method to:
 	- Determine closed-loop pole locations for varying gains
 	- Analyze system stability and performance

### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from control import tf, rlocus, poles, feedback, impulse_response
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact
import warnings

# Open Loop vs Closed Loop Transfer Functions

In control systems engineering, one of the fundamental goals is to understand and manipulate how systems respond to inputs, ensuring that the control objective is achieved. As was introduced in Lecture 5, a key to this understanding lies in the use of transfer functions -- a mathematical description that map inputs to outputs in the frequency domain. 

So far the notion of transfer functions has been introduced. Given an LTI system in its state-space representation, we can transform it to a corresponding transfer function (Lecture 5), and given said transfer function, we know how its poles and zeros affect the output response to test inputs, and can graphically compute the magnitude and phase of the response (Lecture 6). In Lecture 7, the root locus method was introduced as a way of visualizing how the systems' closed-loop poles vary with a feedback gain. Before, introducing the root locus, below we briefly recount the notion of an open/closed-loop system, and the associated conventions that will be used through the remainder of the course. 

## Open Loop System


As introduced in Lecture 1, an open-loop system is a control system where the signals flow in one direction only (See Figure Below). The signals may pass through solely a plant $P(s)$ (the system we are interested in controlling), or there may be a controller $C(s)$ that operates on a reference (input) $r$. The controller aims to help achieve the control objective, and for the time being consider it as pre-processing a reference to help the plant produce more desirable outputs.

Consider the figure below, the controller $C(s)$ receives a reference input $R(s)$ and generates a control signal $u$. This control signal then drives our plant $P(s)$, producing the output $Y(s)$. Notice that at any point, the controller does not know what the output of the system is. 

<div style="text-align:center;">
<img src="./img/OL.png" alt="Open Loop System" width="500">
</div>

The mathematical relationship between input and output is straightforward in this case:
$$L(s) = \frac{Y(s)}{R(s)} = C(s)P(s)$$
In deriving the above, recall that the block diagram algebra introduced in Lecture 1 can be used on transfer functions.

While elegant in its simplicity, this architecture has a critical limitation: it can't compensate for disturbances, uncertainties, or if the system is unstable. Ideally, if there is a mismatch between the reference and output, we would like to account for this dynamically. Thus giving rise to the introduction of feedback and the closed-loop system.


## Closed Loop System


To address these limitations, we can introduce **feedback** by closing-the-signal-loop.

The key difference here is that the output $Y(s)$ is **fed-back** and compared with our reference input. This comparison generates an error signal that tells our controller how far we are from our desired output - a crucial piece of information that was missing in our open loop system.


<div style="text-align:center;">
<img src="./img/CL.png" alt="Closed Loop System" width="500">
</div>


The mathematical description becomes more intricate, but this complexity brings significant benefits. Let's derive the transfer function step by step:

\begin{align}
Y(s) &= P(s)U(s) \\
U(s) &= C(s)E(s) \\
E(s) &= R(s) - Y(s) \\
\end{align}

By substituting these relationships, we can find how the system responds to inputs:
\begin{align}
E(s) &= R(s) - P(s)U(s) \\
E(s) &= R(s) - L(s)E(s) \\
E(s) &= \frac{R(s)}{1 + L(s)}
\end{align}

This leads us to our closed-loop transfer function:
$$T(s) = \frac{Y(s)}{R(s)} = \frac{L(s)}{1 + L(s)}$$

Note that notionally: 
> - $L(s)$ is called the **loop gain**
> - $T(s)$ is called the **Complementary Sensitivity**, the closed loop transfer function
> - $S(s)$ is called the **sensitivity**, the closed-loop transfer function from $r$ to $e$ -- i.e,. $S(s) = \frac{E(s)}{R(s)}$.  

Furthermore, note that although in the above there is only one plant and controller, in general, there could be multiple elements. For exmaple, see the configuration in the figure below. Deriving $L(s)$, $T(s)$ and $S(s)$ for the system below will result in the derived expressions in Lecture 7, slide 22. 

<div style="text-align:center;">
<img src="./img/CL2.png" alt="Closed Loop System" width="500">
</div>


This feedback path fundamentally changes the system's behavior. Now we have control over the system's poles, allowing us to shape its response. When disturbances occur or when the plant's behavior changes, the system detects these deviations and automatically compensates.



# Feedback Control System Behavior: A Deeper Look


Above we introduced the distinction in the transfer function formulation between the open and closed loop systems. To help concretize the effect of a controller, below we provide an example, where, by construction, the open-loop system is unstable. We then closed the loop, and show that a (proportional) controller is capable of stabilizing the system, and derive the controller parameters that ensure the closed-loop system is stable.

The key takeaways from this example should be given an open-loop (potentially unstable system), how can one close the loop and through the use of a controller, derive the controller parameters that ensure stability. 


## Stabilizing the Unstable: The Inverted Pendulum 


Consider the previously visited inverted pendulum problem, with the following parameters: 
- Length: L = $\frac{3}{2}$
- Mass: m = $1$
- Moment of inertia: J = $\frac{1}{3}mL^2$ = $\frac{3}{4}$
- Damping coefficient: c = $\frac{9}{4}$
- Input torque: $\tau_f$

<div style="text-align:center;">
    <img src="./img/pendulum.png" alt="Inverted Pendulum" width="300">
</div>

### The Open-Loop System 
For the time being, we have the open-loop configuration, with no controller:
<div style="text-align:center;">
    <img src="./img/OL_ex.png" alt="Closed loop system example configuration" width="300">
</div>

When the system is linearized around the upright position, the resulting transfer function is:
$$P(s) = \frac{3}{s^2 + 3s - 10}$$

The characteristic equation reveals the system's natural behavior:
$$s^2 + 3s - 10 = 0$$

With poles at:
$$\lambda_1 = 2, \quad \lambda_2 = -5$$

The positive pole ($\lambda_1 = 2$) confirms what is intuitively known - the upright position is unstable since any slight disturbance will cause the pendulum to fall.


### Closing-the-loop

Suppose now we add a proportional feedback controller $C(s) = K$, such that the system architecture is:
<div style="text-align:center;">
    <img src="./img/CL_ex.png" alt="Closed loop system example configuration" width="300">
</div>
Note that in $K$ can be any real value greater than $0$, and it is up to us to select an appropriate $K$ such that the closed-loop system is stable.

The closed-loop transfer function now becomes:
$$T(s) = \frac{3K}{s^2 + 3s - 10 + 3K}$$

With characteristic equation:
$$s^2 + 3s - 10 + 3K = 0$$

And poles:
$$\lambda_{1,2} = \frac{-3 \pm \sqrt{3^2 + 4(10 - 3K)}}{2}$$

We can then determine for what value $K$ the system is stable by ensuring that all poles have non-positive real parts. This is achieved when:
$$K > \frac{10}{3}$$

To better understand the system's behavior and the effect of feedback control, let's explore an interactive visualization. Below, you'll find a dynamic analysis tool that shows both the system's time response and pole locations for different configurations.


In [ ]:
def plot_system_analysis(K, mode):
   # Create figure with two subplots side by side, fixed square size
   fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
   
   # System definition
   num = [3]
   den = [1, 3, -10]
   sys = tf(num, den)
   
   # Plot 1: Impulse Response
   if mode == 'Open-Loop':
       t, y = impulse_response(sys)
       title1 = 'Open-Loop Impulse Response'
       label = 'Open-Loop'
       color = 'green'
       current_sys = sys
   else:
       closed_loop_sys = feedback(K * sys, 1)
       t, y = impulse_response(closed_loop_sys)
       title1 = f'Closed-Loop Impulse Response (K={K:.2f})'
       label = f'Closed-Loop (K={K:.2f})'
       color = 'blue'
       current_sys = closed_loop_sys
   
   ax1.plot(t, y, label=label, color=color, lw=2)
   ax1.set_title(title1)
   ax1.set_xlabel('Time (s)')
   ax1.set_ylabel('Response')
   ax1.grid(True)
   ax1.legend()
   
   # Plot 2: Pole Locations
   poles = current_sys.poles()
   
   # Create pole plot
   ax2.scatter(poles.real, poles.imag, marker='x', s=100, color=color)
   
   # Make the pole plot square with fixed limits
   limit = 10
   ax2.set_xlim(-limit, limit)
   ax2.set_ylim(-limit, limit)
   
   # Add lines for real and imaginary axes
   ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
   ax2.axvline(x=0, color='k', linestyle='-', alpha=0.3)
   
   # Set equal aspect ratio and grid
   ax2.grid(True)
   ax2.set_aspect('equal', adjustable='box')
   
   # Add title and labels
   if mode == 'Open-Loop':
       title2 = 'Open-Loop Pole Locations'
   else:
       title2 = f'Closed-Loop Pole Locations (K={K:.2f})'
   
   ax2.set_title(title2)
   ax2.set_xlabel('Real Part')
   ax2.set_ylabel('Imaginary Part')
   
   # Add labels for poles
   for pole in poles:
       ax2.annotate(f'({pole.real:.2f}, {pole.imag:.2f}j)',
                   (pole.real, pole.imag),
                   xytext=(10, 10), textcoords='offset points')
   
   plt.tight_layout()

def interact_with_system_analysis():
    K_slider = widgets.FloatSlider(
        min=0, max=35, step=0.1, value=1,
        description='Gain (K):'
    )
    
    mode_button = widgets.ToggleButtons(
        options=['Open-Loop', 'Closed-Loop'],
        description='System Mode:',
        disabled=False
    )
    
    # Create an Output widget to hold the K slider
    K_container = widgets.Output()
    
    def on_mode_change(change):
        K_container.clear_output()
        if change['new'] == 'Closed-Loop':
            with K_container:
                display(K_slider)
    
    mode_button.observe(on_mode_change, names='value')
    
    ui = widgets.VBox([mode_button, K_container])
    
    out = widgets.interactive_output(plot_system_analysis, {
        'K': K_slider,
        'mode': mode_button
    })
    
    display(ui, out)

# Call the interactive function
interact_with_system_analysis()

**Left Plot: System Response**
This shows how the system responds to an impulse input over time. In open-loop mode (green), you can observe the unstable behavior we predicted - the pendulum rapidly deviates from equilibrium. Switching to closed-loop mode (blue) demonstrates how feedback affects the output behavior depending on the value of $K$.

**Right Plot: Pole Locations**
The pole plot provides insight into the system's stability characteristics. Recall that poles in the right half-plane (positive real part) indicate instability. In open-loop mode, we can clearly see the unstable pole at +2, confirming our earlier analysis. The closed-loop configuration allows us to move these poles through our choice of gain K.

**Interactive Features:**
- Toggle between 'Open-Loop' and 'Closed-Loop' to compare the two configurations.
- In closed-loop mode, adjust the gain K to see how different feedback strengths affect stability. For what value of $K$ does the closed-loop system become stable? Does this align with the previously derived results?
- Try increasing $K$ gradually in closed-loop mode. Pay close attention to the behavior of the closed-loop poles as $K$ increases, and what occurs in the output response. What do you observe about the relationship between the oscillatory behavior and the position of the poles?

## Dynamic Compensators

To control feedback systems, we as control system designers have one main job: designing the dynamic compensator. This is the controller that we can put between the input and the object we actually want to control, often called the plant. While we will mainly treat these dynamic compensators as transfer functions, they often represent custom analog electronics (in the past) or programs running on a microcontroller.

In the example above we used the simple proportional controller:
$$
C(s) = K
$$
This is of course a very simple dynamic compensator and far from the only one we can use. Two more sophisticated dynamic compensators that we will treat in detail in this course are:
- PID control: 	$$C(s) = K_P + \frac{K_I}{s} + K_Ds$$
- Lead-lag: 	$$C(s) = K \frac{s+z}{s+p}$$

With $G(s) = \frac{1}{s}$ and $G(s) = s$ being the transfer functions of the integrator and differentiator respectively.

To design a dynamic compensator, we have different tools at our disposal. In the following, we will look at one of them, namely the Root Locus Method.

## Root Locus Method

The Root Locus method is a graphical technique in control theory to analyze and design control systems. 
It helps in determining how the poles of the closed-loop system vary as the gain (or another system parameter) is varied. This is important to determine if the system is stable or meets our design criteria.

We will now plot the Root Locus in two different ways.

In the first plot, you can enter an open loop transfer function and then change the gain k and see how that changes the poles and zeros.

In [ ]:
# Global variables to store the last transfer function and previous gain
previous_num = ''
previous_den = ''
previous_gain = 0
pole_trajectories = {}  # Dictionary to store blue points by gain

# Function to compute poles for a given K and open-loop transfer function
def plot_poles_with_K(K, num_str, den_str, window_size):
    global previous_num, previous_den, previous_gain, pole_trajectories

    # Check if the transfer function has changed
    if num_str != previous_num or den_str != previous_den:
        pole_trajectories = {}  # Reset the pole trajectory
        previous_num = num_str  # Update the stored numerator
        previous_den = den_str  # Update the stored denominator

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Closed-loop transfer function for negative feedback
    closed_loop_sys = feedback(K * sys, 1)
    
    # Get poles of the closed-loop system
    poles = closed_loop_sys.poles()
    
    # If K has decreased, remove blue points with gain higher than current K
    if K < previous_gain:
        pole_trajectories = {g: traj for g, traj in pole_trajectories.items() if g <= K}
    
    # Only add new points to the trajectory if K is increasing
    if K > previous_gain:
        pole_trajectories[K] = poles  # Store current poles with gain K as key
    
    # Update the previous gain to the current value of K
    previous_gain = K
    
    # Get all keys (gains) in sorted order for sequential line drawing
    sorted_gains = sorted(pole_trajectories.keys())
    
    # Plot the poles
    plt.figure()
    plt.axhline(0, color='black', lw=1)
    plt.axvline(0, color='black', lw=1)
    
    # Draw lines between each pair of consecutive blue points
    for i in range(len(sorted_gains) - 1):
        gain1, gain2 = sorted_gains[i], sorted_gains[i + 1]
        poles1, poles2 = pole_trajectories[gain1], pole_trajectories[gain2]
        for p1, p2 in zip(poles1, poles2):
            plt.plot([np.real(p1), np.real(p2)], [np.imag(p1), np.imag(p2)], 'b--', alpha=0.5)
    
    # Plot all previous poles as blue points
    all_real_parts = [np.real(p) for traj in pole_trajectories.values() for p in traj]
    all_imag_parts = [np.imag(p) for traj in pole_trajectories.values() for p in traj]
    plt.scatter(all_real_parts, all_imag_parts, color='blue', alpha=0.5)
    
    # Plot the current poles in red
    plt.scatter(np.real(poles), np.imag(poles), color='red', zorder=5, label=f'K = {K}')
    
    # Set plot limits based on the window size slider
    plt.xlim([-window_size, window_size])
    plt.ylim([-window_size, window_size])
    
    plt.grid(True)
    plt.title(f'Poles for K = {K}')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.legend()
    plt.show()

# Interactive function for students to input open-loop transfer function, vary K, and adjust window size
def interact_with_poles():
    global pole_trajectories, previous_gain
    pole_trajectories = {}  # Initialize the trajectory
    previous_gain = 0  # Initialize previous gain to 0
    
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Slider to vary K
    K_slider = widgets.FloatSlider(min=0, max=1000, step=0.1, value=1, description='Gain (K):')
    
    # Slider to adjust the plot window size
    window_slider = widgets.FloatSlider(min=1, max=100, step=0.5, value=10, description='Window Size')
    
    # Combine input boxes, sliders in a vertical layout
    ui = widgets.VBox([num_box, den_box, K_slider, window_slider])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_poles_with_K, {
        'K': K_slider,
        'num_str': num_box,
        'den_str': den_box,
        'window_size': window_slider
    })
    
    # Display the UI and output
    display(ui, out)

# Call the interactive function
interact_with_poles()


If you do this for every k, you will have drawn the root locus. To show that, you can now enter the same transfer function and use the Root Locus function built-in in the Controls library. You will see that every point you plotted above lies on the Root Locus.

In [ ]:
# Function to plot the root locus for the given transfer function
def plot_root_locus(num_str, den_str):
    # Suppress warnings for this function
    warnings.filterwarnings("ignore")

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Plot the root locus
    plt.figure()
    rlocus(sys, plot=True)
    plt.title('Root Locus Plot')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(False)
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.show()

# Interactive function for students to input transfer function and plot the root locus
def interact_with_root_locus():
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Combine input boxes in a vertical layout
    ui = widgets.VBox([num_box, den_box])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_root_locus, {
        'num_str': num_box,
        'den_str': den_box
    })
    
    # Display the UI and output
    display(ui, out)

# Call the function to plot root locus
interact_with_root_locus()

As you see, it is quite easy to draw the root locus on a computer. However, to gain intuition it is important to be able to draw the Root Locus by hand. To do that, there is a certain set of rules:

1. The closed-loop poles need to be symmetric with respect to the real axis (i.e., either real, or complex-conjugate pairs).

2. The number of closed-loop poles is the same as the number of open-loop poles.

3. At $k\rightarrow 0$, the closed-loop poles approach the open-loop poles.

4. At $k\rightarrow \infty$, the closed-loop poles approach the open-loop zeros. Excess closed-loop poles go to infinity.

5. All points on the real axis are on the Root Locus:
	- all points on the real axis to the left of an odd number of poles/zeros are on the positive k Root Locus (the one we usually consider)
	- all points on the real axis to the left of an even number of poles/zeros are on the negative k Root Locus
	
6. If the number of poles is greater than the number of zeros, there are asymptotes:
	- asymptotes intersect the real axis at: $$\sigma = \frac{\Sigma p_i - \Sigma z_i}{\# poles - \# zeros}$$
	- asymptotes radiate out with angle: 	$$\angle s = \frac{180^\circ \pm q \cdot 360^\circ}{\# poles - \# zeros} \text{ if } k>0$$
											$$\angle s = \frac{\pm q \cdot 360^\circ}{\# poles - \# zeros} \text{ if } k<0$$
											with $q = 1, \ldots, (\# poles - \# zeros)$


To get a sense of where these rules come from, you can refer to the lecture or [this video series](https://youtube.com/playlist?list=PLUMWjy5jgHK3-ca6GP6PL0AgcNGHqn33f&si=ba0iUjapBkxisq9B).